In [1]:
import openai
import os, dotenv
from langchain.tools import tool

dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai.api_key = OPENAI_API_KEY

client = openai.OpenAI()

In [2]:
with open("minkhant.txt", "rb") as f:
    file = client.files.create(file=f, purpose="assistants")


In [3]:
vector_store = client.vector_stores.create(
    name = "web_search_test",
    file_ids = [file.id]
)

In [26]:
from datetime import datetime
import random

@tool
def get_current_time() -> str:
    """Get the current time"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location"""
    weather = ['rainy', 'sunny', 'cloudy', 'windy']
    return random.choice(weather)

In [5]:
tool_1 = {
    "type": "function",
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
        },
        "required": ["location"],
    }
}

tool_2 = {
    "type": "function",
    "name": "get_current_time",
    "description": "Get the current time",
    "parameters": {}
}

file_search_tool = {
    "type": "file_search",
    "vector_store_ids": [vector_store.id],
    "max_num_results": 3
}

tools = [tool_1, tool_2, file_search_tool]

In [6]:
name2func = {
    "get_current_weather": get_current_weather,
    "get_current_time": get_current_time,
}

In [7]:
conversation = openai.conversations.create()

In [8]:
input_messages =[{
    "role": "user",
    "content": "Hi!"
}]

In [9]:
simple_response = client.responses.create(
  model="gpt-4.1-mini",
  tools=tools,
  input=input_messages,
  tool_choice="auto",
  conversation=conversation.id
)
print(simple_response)
print(simple_response.output[0].type)

Response(id='resp_0362c46fe8ca495a0068fef326d8f481969f4e582cba381645', created_at=1761538855.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_0362c46fe8ca495a0068fef328c6e48196bb6a0fd3f1f76799', content=[ResponseOutputText(annotations=[], text='Hello! How can I assist you today?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68fef3228060819191d442ded50da853'], filters=None, max_num_results=3, ranking_options=RankingOptions(ranker='auto', score_threshold=0.0)), FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}}, 'required': ['location'], 'additionalProperties': False}, strict=True

In [10]:
input_messages =[{
    "role": "user",
    "content": "What is the weather like in Boston today?"
}]

In [11]:
tool_response = client.responses.create(
  model="gpt-4.1-mini",
  tools=tools,
  input=input_messages,
  tool_choice="auto",
  conversation=conversation.id
)
print(tool_response)
print(tool_response.output[0].type)

Response(id='resp_0362c46fe8ca495a0068fef32a94308196a32f86133d7fdb5a', created_at=1761538858.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseFunctionToolCall(arguments='{"location":"Boston, MA"}', call_id='call_ogUxUtdv9hKBXYxwcfT3vDWf', name='get_current_weather', type='function_call', id='fc_0362c46fe8ca495a0068fef32b4fd88196a666f20e84d088df', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68fef3228060819191d442ded50da853'], filters=None, max_num_results=3, ranking_options=RankingOptions(ranker='auto', score_threshold=0.0)), FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}}, 'required': ['location'], 'additionalProperties': False}, strict=True, type='function', descr

In [12]:
import json
tool_call = tool_response.output[0]
called_tool_name = tool_call.name
called_tool_args = json.loads(tool_call.arguments)

tool_result = name2func[called_tool_name].invoke(input=called_tool_args)
print(tool_result)

windy


In [13]:
# input_messages = [tool_call]
input_messages = []
input_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": tool_result
})

In [14]:
final_response = client.responses.create(
    model="gpt-4.1-mini",
    tools=tools,
    input=input_messages,
    conversation=conversation.id
)
print(final_response)
print(final_response.output[0].type)

Response(id='resp_0362c46fe8ca495a0068fef32c456481968e5e5b78f1ef6835', created_at=1761538860.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_0362c46fe8ca495a0068fef32cf2e481969c286e8150a25c97', content=[ResponseOutputText(annotations=[], text="The weather in Boston today is windy. Is there anything else you'd like to know?", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68fef3228060819191d442ded50da853'], filters=None, max_num_results=3, ranking_options=RankingOptions(ranker='auto', score_threshold=0.0)), FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}}, 'required': ['location'

In [15]:
final_response.output_text

"The weather in Boston today is windy. Is there anything else you'd like to know?"

In [16]:
client.conversations.items.list(conversation.id).to_dict()

{'data': [{'id': 'msg_0362c46fe8ca495a0068fef32cf2e481969c286e8150a25c97',
   'content': [{'annotations': [],
     'text': "The weather in Boston today is windy. Is there anything else you'd like to know?",
     'type': 'output_text',
     'logprobs': []}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'},
  {'id': 'fc_0362c46fe8ca495a0068fef32c8e7c81969d76f67d0a017ae6',
   'call_id': 'call_ogUxUtdv9hKBXYxwcfT3vDWf',
   'output': 'windy',
   'type': 'function_call_output',
   'status': 'completed'},
  {'arguments': '{"location":"Boston, MA"}',
   'call_id': 'call_ogUxUtdv9hKBXYxwcfT3vDWf',
   'name': 'get_current_weather',
   'type': 'function_call',
   'id': 'fc_0362c46fe8ca495a0068fef32b4fd88196a666f20e84d088df',
   'status': 'completed'},
  {'id': 'msg_0362c46fe8ca495a0068fef32ada308196ad04c69029612dfe',
   'content': [{'text': 'What is the weather like in Boston today?',
     'type': 'input_text'}],
   'role': 'user',
   'status': 'completed',
   'type': 'mes

In [17]:
input_messages =[{
    "role": "user",
    "content": "Why do people love Robin Hood?"
}]
file_search_response = client.responses.create(
  model="gpt-4.1-mini",
  tools=tools,
  input=input_messages,
  tool_choice="auto",
  conversation=conversation.id
)
print(file_search_response)
print(file_search_response.output[0].type)
file_search_response.output_text

Response(id='resp_0362c46fe8ca495a0068fef330971081968970794b2f05b5c1', created_at=1761538864.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_0362c46fe8ca495a0068fef33196a4819687c0ce1d7e3fae77', queries=['Why do people love Robin Hood?', 'Robin Hood popularity', 'Robin Hood character traits'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_0362c46fe8ca495a0068fef332e4d88196a61ce8f652a50aa1', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-7k7CLTtQEews58ZXAbsN2Q', filename='minkhant.txt', index=618, type='file_citation')], text='People love Robin Hood because he is a legendary hero known for his bravery, kindness, and sense of justice. He is admired for standing up against injustice by "stealing from the rich and giving to the poor," helping those in need, and inspiring hope among the oppressed. Robi

'People love Robin Hood because he is a legendary hero known for his bravery, kindness, and sense of justice. He is admired for standing up against injustice by "stealing from the rich and giving to the poor," helping those in need, and inspiring hope among the oppressed. Robin Hood\'s cleverness, unmatched skill with the bow and arrow, and his efforts to outwit the cruel Sheriff of Nottingham make him a symbol of fairness and courage. His legend, including his loyalty to his band of Merry Men and his secret acts of heroism, has been told for generations as a reminder to stand up for what is right and help others.'

In [30]:
file_search_response.to_json()

'{\n  "id": "resp_0362c46fe8ca495a0068fef330971081968970794b2f05b5c1",\n  "created_at": 1761538864.0,\n  "error": null,\n  "incomplete_details": null,\n  "instructions": null,\n  "metadata": {},\n  "model": "gpt-4.1-mini-2025-04-14",\n  "object": "response",\n  "output": [\n    {\n      "id": "fs_0362c46fe8ca495a0068fef33196a4819687c0ce1d7e3fae77",\n      "queries": [\n        "Why do people love Robin Hood?",\n        "Robin Hood popularity",\n        "Robin Hood character traits"\n      ],\n      "status": "completed",\n      "type": "file_search_call",\n      "results": null\n    },\n    {\n      "id": "msg_0362c46fe8ca495a0068fef332e4d88196a61ce8f652a50aa1",\n      "content": [\n        {\n          "annotations": [\n            {\n              "file_id": "file-7k7CLTtQEews58ZXAbsN2Q",\n              "filename": "minkhant.txt",\n              "index": 618,\n              "type": "file_citation"\n            }\n          ],\n          "text": "People love Robin Hood because he is a

In [18]:
input_messages =[{
    "role": "user",
    "content": "What is the weather like in New York? What is the today date?"
}]

tools_response = client.responses.create(
  model="gpt-4.1-mini",
  tools=tools,
  input=input_messages,
  tool_choice="auto",
  conversation=conversation.id
)
print(tools_response)

Response(id='resp_0362c46fe8ca495a0068fef565632881969de38261330fb427', created_at=1761539429.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseFunctionToolCall(arguments='{"location":"New York, NY"}', call_id='call_nPTGdZ0sJ8P1FEviK1SVBaUs', name='get_current_weather', type='function_call', id='fc_0362c46fe8ca495a0068fef5666e48819687a69088efb52efa', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68fef3228060819191d442ded50da853'], filters=None, max_num_results=3, ranking_options=RankingOptions(ranker='auto', score_threshold=0.0)), FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}}, 'required': ['location'], 'additionalProperties': False}, strict=True, type='function', des

In [20]:
tools_response.to_json()

'{\n  "id": "resp_0362c46fe8ca495a0068fef565632881969de38261330fb427",\n  "created_at": 1761539429.0,\n  "error": null,\n  "incomplete_details": null,\n  "instructions": null,\n  "metadata": {},\n  "model": "gpt-4.1-mini-2025-04-14",\n  "object": "response",\n  "output": [\n    {\n      "arguments": "{\\"location\\":\\"New York, NY\\"}",\n      "call_id": "call_nPTGdZ0sJ8P1FEviK1SVBaUs",\n      "name": "get_current_weather",\n      "type": "function_call",\n      "id": "fc_0362c46fe8ca495a0068fef5666e48819687a69088efb52efa",\n      "status": "completed"\n    }\n  ],\n  "parallel_tool_calls": true,\n  "temperature": 1.0,\n  "tool_choice": "auto",\n  "tools": [\n    {\n      "type": "file_search",\n      "vector_store_ids": [\n        "vs_68fef3228060819191d442ded50da853"\n      ],\n      "filters": null,\n      "max_num_results": 3,\n      "ranking_options": {\n        "ranker": "auto",\n        "score_threshold": 0.0\n      }\n    },\n    {\n      "name": "get_current_weather",\n      

In [21]:
tool_call = tools_response.output[0]
called_tool_name = tool_call.name
called_tool_args = json.loads(tool_call.arguments)

tool_result = name2func[called_tool_name].invoke(input=called_tool_args)
print(tool_result)

cloudy


In [22]:
# input_messages = [tool_call]
input_messages = []
input_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": tool_result
})

next_response = client.responses.create(
    model="gpt-4.1-mini",
    tools=tools,
    input=input_messages,
    conversation=conversation.id
)
print(next_response)
print(next_response.output[0].type)

Response(id='resp_0362c46fe8ca495a0068fef6f984d88196bb0fa8ae26e03bad', created_at=1761539834.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseFunctionToolCall(arguments='{}', call_id='call_4svmvQl53da9kK6QwGTrOOBl', name='get_current_time', type='function_call', id='fc_0362c46fe8ca495a0068fef6fb50908196a2b6c9f70ac8ce91', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68fef3228060819191d442ded50da853'], filters=None, max_num_results=3, ranking_options=RankingOptions(ranker='auto', score_threshold=0.0)), FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}}, 'required': ['location'], 'additionalProperties': False}, strict=True, type='function', description='Get the current we

In [23]:
next_response.to_json()

'{\n  "id": "resp_0362c46fe8ca495a0068fef6f984d88196bb0fa8ae26e03bad",\n  "created_at": 1761539834.0,\n  "error": null,\n  "incomplete_details": null,\n  "instructions": null,\n  "metadata": {},\n  "model": "gpt-4.1-mini-2025-04-14",\n  "object": "response",\n  "output": [\n    {\n      "arguments": "{}",\n      "call_id": "call_4svmvQl53da9kK6QwGTrOOBl",\n      "name": "get_current_time",\n      "type": "function_call",\n      "id": "fc_0362c46fe8ca495a0068fef6fb50908196a2b6c9f70ac8ce91",\n      "status": "completed"\n    }\n  ],\n  "parallel_tool_calls": true,\n  "temperature": 1.0,\n  "tool_choice": "auto",\n  "tools": [\n    {\n      "type": "file_search",\n      "vector_store_ids": [\n        "vs_68fef3228060819191d442ded50da853"\n      ],\n      "filters": null,\n      "max_num_results": 3,\n      "ranking_options": {\n        "ranker": "auto",\n        "score_threshold": 0.0\n      }\n    },\n    {\n      "name": "get_current_weather",\n      "parameters": {\n        "type": "ob

In [28]:
tool_call = next_response.output[0]
called_tool_name = tool_call.name
called_tool_args = json.loads(tool_call.arguments)

tool_result = name2func[called_tool_name].invoke(input=called_tool_args)

input_messages = []
input_messages.append({
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": tool_result
})

final_response2 = client.responses.create(
    model="gpt-4.1-mini",
    tools=tools,
    input=input_messages,
    conversation=conversation.id
)
print(final_response2)
print(final_response2.output[0].type)

Response(id='resp_0362c46fe8ca495a0068fef7a4f384819699a6acbfbd401cc5', created_at=1761540005.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-mini-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_0362c46fe8ca495a0068fef7a64c988196af1dcb0d3ebb24ab', content=[ResponseOutputText(annotations=[], text="The weather in New York today is cloudy. The date today is October 27, 2025. Is there anything else you'd like to know?", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68fef3228060819191d442ded50da853'], filters=None, max_num_results=3, ranking_options=RankingOptions(ranker='auto', score_threshold=0.0)), FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Fr

In [29]:
final_response2.to_json()

'{\n  "id": "resp_0362c46fe8ca495a0068fef7a4f384819699a6acbfbd401cc5",\n  "created_at": 1761540005.0,\n  "error": null,\n  "incomplete_details": null,\n  "instructions": null,\n  "metadata": {},\n  "model": "gpt-4.1-mini-2025-04-14",\n  "object": "response",\n  "output": [\n    {\n      "id": "msg_0362c46fe8ca495a0068fef7a64c988196af1dcb0d3ebb24ab",\n      "content": [\n        {\n          "annotations": [],\n          "text": "The weather in New York today is cloudy. The date today is October 27, 2025. Is there anything else you\'d like to know?",\n          "type": "output_text",\n          "logprobs": []\n        }\n      ],\n      "role": "assistant",\n      "status": "completed",\n      "type": "message"\n    }\n  ],\n  "parallel_tool_calls": true,\n  "temperature": 1.0,\n  "tool_choice": "auto",\n  "tools": [\n    {\n      "type": "file_search",\n      "vector_store_ids": [\n        "vs_68fef3228060819191d442ded50da853"\n      ],\n      "filters": null,\n      "max_num_results":